<a href="https://colab.research.google.com/github/pawarad/PlasmidGPT/blob/main/PlasmidGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# read it in to inspect it
with open('plasmid_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [10]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  82837504


In [11]:
# let's look at the first 1000 characters
print(text[:1000])

>Escherichia coli strain B20123 plasmid pB20123_4, complete sequence
ATGAGCGCCGCGCTTCAATACTTCGACGAAAATTTACCCCATCGTCCCTATCACACGGATGATCTCGCTTTTGGTCTTCGCATCTCCGGCAAAGGGCGTGCGCTTCTTGCGCGGTACATACAGCAGAACCAGCCTCATGCGCAGTTCTGGCTGGTTTTTGACGTTGACCGCGAGGGGGCTGCGATTGACTGGAGCGACCGGAACGCTCCCGCGCCGAACATCACCGTTAAAAATCCTGTGAACGGACACGCTCACCTGCTCTACGCACTCAACATCGCCGTGAGAACCGCGCCTGATGCCTCGGTTAAGGCGCTGAAATATGCCGCAGCGGTTGAACGTTCGCTGTGTGAGAAACTTTGTGCGGATGTGAACTACAGCGGCCTGATTTGCAAAAATCCGTTCCACCTGGAATGGCTGGTGATGGAGTGGCGCGAGGAAGCCTATACCCTCGATGAACTGGCTGATTATCTCGATTTGAGCGCCTCAGAGCGTCGTAGCATCGATAAACATTACGGGATGGGGCGAAACTGCCACCTGTTCGAAATGACGCGCAAATGGGCTTACAGGGCGATTCGTCAGGGCTGGCCAGCGTTCTCACAGTGGCTTGAAGCGGTGATCCAGCGTGTCGAAATGTACAATGCATCGCTTCCCGTTCCGCTTTCACTGGCTGAATGTCGGGCTATTGGCAAGAGCATTGCGAAATACACGCACAGGAACTTCACGCCGGAAACTTTCGCACAGTATGTGGCTGATACGCACACGCCAGAAATTCAGGCTGCACGCGGTCGCAAGGGCGGTTCTAAGTCTAAGCGCAGCACAGTAGCTACATCTGCACGCACGCTGAAACCGTGGGAGGCATTAGGGATCAGTCGCGCCTGGTATTACCAACTGAAAAAACGAGGTCTCGTAGAGTAGACCAAATAAGCCTATA

In [12]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 ,-./0123456789:>ABCDEFGHIJKLMNOPRSTUVYZ_abcdefghiklmnopqrstuvwxyz
67


In [13]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("Escherichia coli strain B20123 plasmid pB20123_4, complete sequence"))
print(decode(encode("Escherichia coli strain B20123 plasmid pB20123_4, complete sequence")))

[22, 59, 44, 49, 46, 58, 50, 44, 49, 50, 42, 1, 44, 55, 52, 50, 1, 59, 60, 58, 42, 50, 54, 1, 19, 8, 6, 7, 8, 9, 1, 56, 52, 42, 59, 53, 50, 45, 1, 56, 19, 8, 6, 7, 8, 9, 41, 10, 2, 1, 44, 55, 53, 56, 52, 46, 60, 46, 1, 59, 46, 57, 61, 46, 54, 44, 46]
Escherichia coli strain B20123 plasmid pB20123_4, complete sequence


In [14]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([82837504]) torch.int64
tensor([17, 22, 59, 44, 49, 46, 58, 50, 44, 49, 50, 42,  1, 44, 55, 52, 50,  1,
        59, 60, 58, 42, 50, 54,  1, 19,  8,  6,  7,  8,  9,  1, 56, 52, 42, 59,
        53, 50, 45,  1, 56, 19,  8,  6,  7,  8,  9, 41, 10,  2,  1, 44, 55, 53,
        56, 52, 46, 60, 46,  1, 59, 46, 57, 61, 46, 54, 44, 46,  0, 18, 36, 24,
        18, 24, 20, 24, 20, 20, 24, 20, 24, 20, 36, 36, 20, 18, 18, 36, 18, 20,
        36, 36, 20, 24, 18, 20, 24, 18, 18, 18, 18, 36, 36, 36, 18, 20, 20, 20,
        20, 18, 36, 20, 24, 36, 20, 20, 20, 36, 18, 36, 20, 18, 20, 18, 20, 24,
        24, 18, 36, 24, 18, 36, 20, 36, 20, 24, 20, 36, 36, 36, 36, 24, 24, 36,
        20, 36, 36, 20, 24, 20, 18, 36, 20, 36, 20, 20, 24, 24, 20, 18, 18, 18,
        24, 24, 24, 20, 24, 36, 24, 20, 24, 20, 36, 36, 20, 36, 36, 24, 20, 24,
        20, 24, 24, 36, 18, 20, 18, 36, 18, 20, 18, 24, 20, 18, 24, 18, 18, 20,
        20, 18, 24, 20, 20, 36, 20, 18, 36, 24, 20, 24, 20, 18, 24, 36, 36, 20,
     

In [15]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [16]:
block_size = 8
train_data[:block_size+1]

tensor([17, 22, 59, 44, 49, 46, 58, 50, 44])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([17]) the target: 22
when input is tensor([17, 22]) the target: 59
when input is tensor([17, 22, 59]) the target: 44
when input is tensor([17, 22, 59, 44]) the target: 49
when input is tensor([17, 22, 59, 44, 49]) the target: 46
when input is tensor([17, 22, 59, 44, 49, 46]) the target: 58
when input is tensor([17, 22, 59, 44, 49, 46, 58]) the target: 50
when input is tensor([17, 22, 59, 44, 49, 46, 58, 50]) the target: 44


In [18]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[36, 24, 18, 18, 24, 36, 36, 18],
        [36, 24, 36, 18, 36, 20, 18, 18],
        [36, 20, 20, 20, 36, 36, 36, 18],
        [36, 36, 36, 24, 36, 20, 24, 18]])
targets:
torch.Size([4, 8])
tensor([[24, 18, 18, 24, 36, 36, 18, 24],
        [24, 36, 18, 36, 20, 18, 18, 24],
        [20, 20, 20, 36, 36, 36, 18, 18],
        [36, 36, 24, 36, 20, 24, 18, 36]])
----
when input is [36] the target: 24
when input is [36, 24] the target: 18
when input is [36, 24, 18] the target: 18
when input is [36, 24, 18, 18] the target: 24
when input is [36, 24, 18, 18, 24] the target: 36
when input is [36, 24, 18, 18, 24, 36] the target: 36
when input is [36, 24, 18, 18, 24, 36, 36] the target: 18
when input is [36, 24, 18, 18, 24, 36, 36, 18] the target: 24
when input is [36] the target: 24
when input is [36, 24] the target: 36
when input is [36, 24, 36] the target: 18
when input is [36, 24, 36, 18] the target: 36
when input is [36, 24, 36, 18, 36] the target: 20
when inp

In [19]:
print(xb) # our input to the transformer

tensor([[36, 24, 18, 18, 24, 36, 36, 18],
        [36, 24, 36, 18, 36, 20, 18, 18],
        [36, 20, 20, 20, 36, 36, 36, 18],
        [36, 36, 36, 24, 36, 20, 24, 18]])


In [20]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 67])
tensor(4.8664, grad_fn=<NllLossBackward0>)

w/eV9oeT2S2R,UK:
MAy.:wUryu6-v>Ev6PCERvE.hmrBs1w DMBAsoRzAdOIuef>LYwr>YNbKNDUPBtyEn9ixa4J5weKR6uL26M


In [21]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [22]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.73695707321167


In [23]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


TTnc9A.ZCFf13v69oPfZTHCsVa69GLIm>spglwF/sl2NbvgP4dB6
iOyGFAf8q3p3vb/k,.nx2spk.OgyfoE8VU7eK_gP>Juk>Ew vAucO,KD_GKxb
.S:aSqBv>RH
v
r90A>J7a28bReR-JR:wFF6-bsVg0BuzFR0D/IB_LwfMPP5YT2N.uBmCE-rO8V9dHMyYqz5 2U-J>lL6-YNpFFf 42GJ6y.N7oRO179b_xMP1wroLo8v-Z4rvecd8tO8SNCYbz0VnMmN:kR67bR1_Tl2Rb_wGv_ 35vH.D_fu/d,xO87waY1NC31xZk,v5Gih9aJS6-u98.2fMdO8Bg89YNkx3vE_cdRDuBqdRLoaNr3tfM3IU9gegTSiKBrnMP4ezCPd,5G16aA3Im>_fvMCzYVHqNP-Yby2sCaA8CE_th6AOkJqNby_l2ttr
Bm4JtE 7w_tO4aigzI_ABRbvtntEf>xV
72RzFIToa_y0Lmm/Z8xnZSV



In [24]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [25]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [26]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [27]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [28]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [29]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [30]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [31]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [32]:
k.var()

tensor(1.0449)

In [33]:
q.var()

tensor(1.0700)

In [34]:
wei.var()

tensor(1.0918)

In [35]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [36]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [37]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [38]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [39]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('plasmid_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209987 M parameters
step 0: train loss 4.2950, val loss 4.2990
step 100: train loss 1.3824, val loss 1.3657
step 200: train loss 1.3713, val loss 1.3561
step 300: train loss 1.3679, val loss 1.3576
step 400: train loss 1.3657, val loss 1.3653
step 500: train loss 1.3653, val loss 1.3568
step 600: train loss 1.3643, val loss 1.3541
step 700: train loss 1.3681, val loss 1.3527
step 800: train loss 1.3618, val loss 1.3486
step 900: train loss 1.3617, val loss 1.3517
step 1000: train loss 1.3745, val loss 1.3625
step 1100: train loss 1.3619, val loss 1.3446
step 1200: train loss 1.3613, val loss 1.3594
step 1300: train loss 1.3558, val loss 1.3464
step 1400: train loss 1.3597, val loss 1.3505
step 1500: train loss 1.3667, val loss 1.3495
step 1600: train loss 1.3572, val loss 1.3436
step 1700: train loss 1.3591, val loss 1.3524
step 1800: train loss 1.3550, val loss 1.3436
step 1900: train loss 1.3550, val loss 1.3436
step 2000: train loss 1.3538, val loss 1.3434
step 2100: train loss 1.

In [41]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


TCATACCTAACTTGGGGCACGTCTAATTGTTAGACACATTGCAACGCGCAAAAGCGGAATATCTGGGGTGTCGCGCGCGAAAACGACTTCCGCCAGTAAAAAGGCTTCAAGACGGGCCACGATGAATAAAAAGTGGCGTGTTTGCTTTTTTTTTTACTTTTCCGTCTATCTGCACAGTTCAAACGATATCTCATGCTGATATCAAGAATGTAACCCGCTCAGACGATACAAAGGCGCCGGCCAAGGCACCGTTTACCTCCCCGCGGTTTTCAGATTCGATTTCCTGTCATCCTTCCTCCCGCATAGCCATTATCACCCTGGCGCGCGTACCAGTATTTTAAATTCTGAACGCACCGATCGTACCTTTTCCGGTAGACCCACTCCGGTGAACGATGCCGGTAGCTCGTGAAAGCGTGACAGGCTGATTCAGCAGCTTTCAGCAATGACGCCGCCTGGCGTAAAAATCTCCGCCCCGTGCTGGCCGTCGAGCCGCGCTGCTC


In [43]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=50)[0].tolist()))


CTTTTCCCCATCAGCTTCGACGGCGCTAGGGCGCGGTCATGTGCCTCGCG
